## Introduction to Quantum Matcha Tea backend in QiboTN

#### Some imports

In [1]:
import time
import numpy as np
from scipy import stats

import qibo
from qibo import Circuit, gates, hamiltonians
from qibo.backends import construct_backend

#### Some hyper parameters

In [2]:
# construct qibotn backend
qmatcha_backend = construct_backend(backend="qibotn", platform="qmatchatea")

# set number of qubits
nqubits = 4

# set numpy random seed
np.random.seed(42)

#### Constructing a parametric quantum circuit

In [3]:
def build_circuit(nqubits, nlayers):
    """Construct a parametric quantum circuit."""
    circ = Circuit(nqubits)
    for _ in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
        [circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    circ.add(gates.M(*range(nqubits)))
    return circ

In [4]:
circuit = build_circuit(nqubits=nqubits, nlayers=3)
circuit.draw()

0: ─RY─RZ─o─────X─RY─RZ─o─────X─RY─RZ─o─────X─M─
1: ─RY─RZ─X─o───|─RY─RZ─X─o───|─RY─RZ─X─o───|─M─
2: ─RY─RZ───X─o─|─RY─RZ───X─o─|─RY─RZ───X─o─|─M─
3: ─RY─RZ─────X─o─RY─RZ─────X─o─RY─RZ─────X─o─M─


In [5]:
# Setting random parameters
circuit.set_parameters(
    parameters=np.random.uniform(-np.pi, np.pi, len(circuit.get_parameters())),
)

#### Setting up the tensor network simulator

Depending on the simulator, various parameters can be set. One can customize the tensor network execution via the `backend.configure_tn_simulation` function, whose face depends on the specific backend provider.

In [6]:
# Customization of the tensor network simulation in the case of qmatchatea
# Here we use only some of the possible arguments
qmatcha_backend.configure_tn_simulation(
    ansatz="MPS",
    max_bond_dimension=10,
    cut_ratio=1e-6,
)

#### Executing through the backend

The `backend.execute_circuit` method can be used then. We can simulate results in three ways:
1. reconstruction of the final state (statevector like, only if `nqubits < 20` due to Quantum Matcha Tea setup) only if `return_array` is set to `True`;
2. computation of the relevant probabilities of the final state. There are three way of doing so, but we will see it directly into the docstrings;
3. reconstruction of the relevant state's frequencies (only if `nshots` is not `None`).

In [7]:
# Simple execution (defaults)
outcome = qmatcha_backend.execute_circuit(circuit=circuit)

# Print outcome
vars(outcome)

{'nqubits': 4,
 'backend': QMatchaTeaBackend(),
 'measures': None,
 'measured_probabilities': {'U': {'0000': (0.0, 0.08390937969317301),
   '0001': (0.08390937969317301, 0.08858639088838134),
   '0010': (0.08858639088838131, 0.1832549957082757),
   '0011': (0.1832549957082757, 0.25896776804349736),
   '0100': (0.2589677680434974, 0.33039716334036867),
   '0101': (0.33039716334036867, 0.386620221067355),
   '0110': (0.3866202210673549, 0.4380808691410473),
   '0111': (0.4380808691410473, 0.47837271988834),
   '1000': (0.47837271988834, 0.5916815553716759),
   '1001': (0.5916815553716759, 0.5972581739037379),
   '1010': (0.5972581739037378, 0.6359857590550054),
   '1011': (0.6359857590550054, 0.6894851559808782),
   '1100': (0.6894851559808783, 0.7030911408535467),
   '1101': (0.7030911408535467, 0.8264027395524797),
   '1110': (0.8264027395524797, 0.8981519382820797),
   '1111': (0.8981519382820797, 0.9999999999999998)},
  'E': [None],
  'G': [None]},
 'prob_type': 'U',
 'statevector': 

In [8]:
# Execution with a specific probability type
# We use here "E", which is cutting some of the components if under a threshold
# We also retrieve the statevector
outcome = qmatcha_backend.execute_circuit(
    circuit=circuit,
    prob_type="G",
    prob_threshold=0.3,
    return_array=True,
)

# Print outcome
vars(outcome)

{'nqubits': 4,
 'backend': QMatchaTeaBackend(),
 'measures': None,
 'measured_probabilities': {'U': [None],
  'E': [None],
  'G': {'1110': 0.07174919872960005,
   '1111': 0.10184806171792007,
   '0010': 0.09466860481989439,
   '0011': 0.07571277233522165}},
 'prob_type': 'G',
 'statevector': array([ 0.08809627-0.27595005j,  0.24859731-0.22695421j,
         0.18807826+0.18988408j,  0.09444097+0.06846085j,
         0.00470148+0.30764671j,  0.17371395-0.09247188j,
        -0.18900305+0.12545316j, -0.17359753+0.20399288j,
        -0.0517478 +0.04471215j, -0.0411739 -0.06230031j,
         0.22377064+0.07842041j, -0.21784975-0.27541439j,
        -0.27208941+0.04098933j, -0.22748127+0.04185292j,
         0.17105258-0.10503745j, -0.01729753-0.31866731j])}

---

One can access to the specific contents of the simulation outcome.

In [9]:
print(f"Probabilities:\n {outcome.probabilities()}\n")
print(f"State:\n {outcome.state()}\n")

Probabilities:
 [0.0717492  0.10184806 0.0946686  0.07571277]

State:
 [ 0.08809627-0.27595005j  0.24859731-0.22695421j  0.18807826+0.18988408j
  0.09444097+0.06846085j  0.00470148+0.30764671j  0.17371395-0.09247188j
 -0.18900305+0.12545316j -0.17359753+0.20399288j -0.0517478 +0.04471215j
 -0.0411739 -0.06230031j  0.22377064+0.07842041j -0.21784975-0.27541439j
 -0.27208941+0.04098933j -0.22748127+0.04185292j  0.17105258-0.10503745j
 -0.01729753-0.31866731j]



---

But frequencies cannot be accessed, since no shots have been set.

---

We can then repeat the execution by setting the number of shots

In [10]:
# Execution with a specific probability type
# We use here "E", which is cutting some of the components if under a threshold
outcome = qmatcha_backend.execute_circuit(
    circuit=circuit,
    nshots=1024,
    prob_type="E",
    prob_threshold=0.05,
    return_array=True
)

# Print outcome
vars(outcome)

{'nqubits': 4,
 'backend': QMatchaTeaBackend(),
 'measures': {'0000': 92,
  '0001': 7,
  '0010': 85,
  '0011': 79,
  '0100': 81,
  '0101': 55,
  '0110': 47,
  '0111': 39,
  '1000': 117,
  '1001': 7,
  '1010': 38,
  '1011': 53,
  '1100': 22,
  '1101': 129,
  '1110': 74,
  '1111': 99},
 'measured_probabilities': {'U': [None],
  'E': {'0000': 0.08390937969317301,
   '0010': 0.09466860481989439,
   '0011': 0.07571277233522165,
   '0100': 0.07142939529687124,
   '0101': 0.05622305772698632,
   '0110': 0.05146064807369245,
   '1000': 0.11330883548333581,
   '1011': 0.053499396925872765,
   '1101': 0.12331159869893296,
   '1110': 0.07174919872960005,
   '1111': 0.10184806171792007},
  'G': [None]},
 'prob_type': 'E',
 'statevector': array([ 0.08809627-0.27595005j,  0.24859731-0.22695421j,
         0.18807826+0.18988408j,  0.09444097+0.06846085j,
         0.00470148+0.30764671j,  0.17371395-0.09247188j,
        -0.18900305+0.12545316j, -0.17359753+0.20399288j,
        -0.0517478 +0.04471215j, 

In [11]:
# Frequencies and probabilities
print(f"Frequencies:\n {outcome.frequencies()}\n")
print(f"Probabilities:\n {outcome.probabilities()}\n")
print(f"State:\n {outcome.state()}\n")  # Only if return_array = True

Frequencies:
 {'0000': 92, '0001': 7, '0010': 85, '0011': 79, '0100': 81, '0101': 55, '0110': 47, '0111': 39, '1000': 117, '1001': 7, '1010': 38, '1011': 53, '1100': 22, '1101': 129, '1110': 74, '1111': 99}

Probabilities:
 [0.08390938 0.0946686  0.07571277 0.0714294  0.05622306 0.05146065
 0.11330884 0.0534994  0.1233116  0.0717492  0.10184806]

State:
 [ 0.08809627-0.27595005j  0.24859731-0.22695421j  0.18807826+0.18988408j
  0.09444097+0.06846085j  0.00470148+0.30764671j  0.17371395-0.09247188j
 -0.18900305+0.12545316j -0.17359753+0.20399288j -0.0517478 +0.04471215j
 -0.0411739 -0.06230031j  0.22377064+0.07842041j -0.21784975-0.27541439j
 -0.27208941+0.04098933j -0.22748127+0.04185292j  0.17105258-0.10503745j
 -0.01729753-0.31866731j]



### Compute expectation values

Another important feature of this backend is the `expectation` function. In fact, we can compute expectation values of given observables thorugh a Qibo-friendly interface.

---

Let's start by importing some symbols, thanks to which we can build our observable.

In [12]:
from qibo.symbols import Z, X

In [13]:
# We are going to compute the expval of an Hamiltonian
# On the state prepared by the following circuit
circuit.draw()

circuit.set_parameters(
    np.random.randn(len(circuit.get_parameters()))
)

0: ─RY─RZ─o─────X─RY─RZ─o─────X─RY─RZ─o─────X─M─
1: ─RY─RZ─X─o───|─RY─RZ─X─o───|─RY─RZ─X─o───|─M─
2: ─RY─RZ───X─o─|─RY─RZ───X─o─|─RY─RZ───X─o─|─M─
3: ─RY─RZ─────X─o─RY─RZ─────X─o─RY─RZ─────X─o─M─


In [14]:
# We can create a symbolic Hamiltonian
form = 0.5 * Z(0) * Z(1) +- 1.5 *  X(0) * Z(2) + Z(3)
hamiltonian = hamiltonians.SymbolicHamiltonian(form)

#  Let's show it
hamiltonian.form

[Qibo 0.2.15|INFO|2025-02-12 14:36:17]: Using qibojit (numba) backend on /CPU:0


-1.5*X0*Z2 + 0.5*Z0*Z1 + Z3

In [15]:
# And compute its expectation value
qmatcha_backend.expectation(
    circuit=circuit,
    observable=hamiltonian,
)

0.4355195352502318

In [16]:
# Try with Qibo (which is by default using the Qibojit backend)
hamiltonian = hamiltonians.SymbolicHamiltonian(form)
hamiltonian.expectation(circuit().state())

0.43551953525022985

They match! 🥳